# Рекомендация тарифов

#### Описание проекта
Многие клиенты мобильного оператора пользуются архивными тарифами. Оператор хочет предложить им один из новых тарифов. Чтобы предложить клиенту архивного тарифа подходящий новый, необходимо научится по поведению клиентов новых тарифов определить, каким тарифом они пользуются.  
#### Цель
Имеются данные о поведении клиентов, которые уже перешли на новые тарифы. Необходимо построить модель для задачи классификации, которая выберет подходящий тариф.
#### Задача
Построить модель с максимально большим значением **accuracy**, но не меньше 0,75.

### Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.  
Известно:  
сalls — количество звонков,  
minutes — суммарная длительность звонков в минутах,  
messages — количество sms-сообщений,  
mb_used — израсходованный интернет-трафик в Мб,  
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

### Описание тарифов
#### Тариф «Смарт»
Ежемесячная плата: 550 рублей  
Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика  
Стоимость услуг сверх тарифного пакета:  
минута разговора: 3 рубля  
сообщение: 3 рубля  
1 Гб интернет-трафика: 200 рублей  
#### Тариф «Ультра»
Ежемесячная плата: 1950 рублей  
Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика  
Стоимость услуг сверх тарифного пакета:  
минута разговора: 1 рубль  
сообщение: 1 рубль  
1 Гб интернет-трафика: 150 рублей  
  
Оператор всегда округляет вверх значения минут и мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.

## 1. Открытие файла

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# открываем таблицу с информацией о пользователях
data_users = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data_users.info()
data_users.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Пусть данные и предобработаны, но все равно приведем значения к типу int с округлением в большую сторону, а мегабайты переведем в гигобайты.

In [4]:
# приведем к типу int с округлением в большую сторону
data_users['calls'] = data_users['calls'].astype(int)
data_users['minutes'] = np.ceil(data_users['minutes']).astype(int)
data_users['messages'] = data_users['messages'].astype(int)
# заодно переведем мегабайты в гигобайты
data_users['mb_used'] = np.ceil(data_users['mb_used'] / 1024).astype(int)
# переименуем столбец mb_used в пb_used
data_users.rename(columns={'mb_used': 'gb_used'}, inplace=True)

In [5]:
data_users.head()

,calls,minutes,messages,gb_used,is_ultra
0,40,312,83,20,0
1,85,517,56,23,0
2,77,468,86,21,0
3,106,746,81,9,1
4,66,419,1,15,0


#### Далее следует некоторое исправление данных.  
Так как люди могут ошибаться в выборе тариф, то поменяем в данных некоторые значения в столбце **is_ultra** по следующему принципу:  
*  абонентов тафира ultra переведем на тариф smart в случае, если количество израсходованных минут, отправленных сообщений и использованных объемов интернет-трафика меньше, чем включено в тариф smart.  
*  абонентов тафира smart переведем на тариф ultra в случае, если их месячный платеж окажется больше, чем на тарифе ultra.

Создаем ключевой признак **is_ultra_corrected**, в котором будем исправлять неверно выбранный тариф

In [6]:
data_users['is_ultra_corrected'] = data_users['is_ultra']

Подсчитаем, сколько абонентов надо перевести с тарифа smart на ultra

In [7]:
data_users.query('minutes < 500 and messages < 50 and gb_used < 15 and is_ultra == 1').count()

calls                 138
minutes               138
messages              138
gb_used               138
is_ultra              138
is_ultra_corrected    138
dtype: int64

138 абонентов могли бы спокойно пользоваться тарифом smart

Меняем тафир ultra на smart

In [8]:
data_users.loc[(data_users['minutes'] < 500) & (data_users['messages'] < 50) & (data_users['gb_used'] < 15) & (data_users['is_ultra_corrected'] == 1), 'is_ultra_corrected'] = 0

Задаем функцию, котора посчитает месячную выручку с каждого абонента

In [9]:
# задаем функцию
def money_counter (tariff, duration, messages, gb_used):
    if tariff == 0:
        price_per_month = 550
        duration_limit = 500
        massages_limit = 50
        gb_limit = 15
        pay_for_duration_overlimit = 3
        pay_for_massages_overlimit = 3
        pay_for_gb_overlimit = 200
    if tariff == 1:
        price_per_month = 1950
        duration_limit = 3000
        massages_limit = 1000
        gb_limit = 30
        pay_for_duration_overlimit = 1
        pay_for_massages_overlimit = 1
        pay_for_gb_overlimit = 150
    duration_finally = duration - duration_limit
    messages_finally = messages - massages_limit
    gb_finally = gb_used - gb_limit
    if (duration_finally < 0):
        duration_finally = 0
    if (messages_finally < 0):
        messages_finally = 0
    if (gb_finally < 0):
        gb_finally = 0
    pay_for_month = (price_per_month
                         + duration_finally * pay_for_duration_overlimit
                         + messages_finally * pay_for_massages_overlimit
                         + gb_finally * pay_for_gb_overlimit)
    return pay_for_month

Считаем месячную выручку с абонента

In [10]:
for i in range(len(data_users)):
    data_users.loc[i, 'pay_per_month'] = (money_counter (data_users.loc[i, 'is_ultra_corrected'], 
                                                         data_users.loc[i, 'minutes'], 
                                                         data_users.loc[i, 'messages'],
                                                         data_users.loc[i, 'gb_used']))
data_users.head()

,calls,minutes,messages,gb_used,is_ultra,is_ultra_corrected,pay_per_month
0,40,312,83,20,0,0,1649.0
1,85,517,56,23,0,0,2219.0
2,77,468,86,21,0,0,1858.0
3,106,746,81,9,1,1,1950.0
4,66,419,1,15,0,0,550.0


Подсчитаем количество людей, которым было бы лучше перейти на тариф ultra

In [11]:
data_users.query('pay_per_month >= 1950 and is_ultra_corrected == 0').count()

calls                 460
minutes               460
messages              460
gb_used               460
is_ultra              460
is_ultra_corrected    460
pay_per_month         460
dtype: int64

Таких оказалось 460 человек.  
Меняем тариф со smart на ultra в этих случаях

In [12]:
data_users.loc[(data_users['pay_per_month'] >= 1950) & (data_users['is_ultra_corrected'] == 0), 'is_ultra_corrected'] = 1

## 2. Разбиение данных на выборки

Разделите исходные данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1.  
Определим функцию, которая будет разделять выборки.

In [13]:
# Деление на обучающую + валидационную и тестовую выборки.
data_users_tr_val, data_users_test = train_test_split(data_users, test_size=0.2, random_state=12345)
# Деление обучающей + валидационной выборки на обучающую и валидационную выборки
data_users_trainig, data_users_valid = train_test_split(data_users_tr_val, test_size=0.25, random_state=12345)
# Разделение выборки на выборки с признаками (без столбца is_ultra) и выборки с ключевым признаком (только столбец is_ultra)
features_data_users_tr_val = data_users_tr_val.drop(['is_ultra_corrected', 'is_ultra', 'pay_per_month'], axis=1)
target_data_users_tr_val = data_users_tr_val['is_ultra']
c_target_data_users_tr_val = data_users_tr_val['is_ultra_corrected']
# тестовые выборки
features_data_users_test = data_users_test.drop(['is_ultra_corrected', 'is_ultra', 'pay_per_month'], axis=1)
target_data_users_test = data_users_test['is_ultra']
c_target_data_users_test = data_users_test['is_ultra_corrected']
# тренировочные выборки
features_data_users_trainig = data_users_trainig.drop(['is_ultra_corrected', 'is_ultra', 'pay_per_month'], axis=1)
target_data_users_trainig = data_users_trainig['is_ultra']
c_target_data_users_trainig = data_users_trainig['is_ultra_corrected']
# валидационные выборки
features_data_users_valid = data_users_valid.drop(['is_ultra_corrected', 'is_ultra', 'pay_per_month'], axis=1)
target_data_users_valid = data_users_valid['is_ultra']
c_target_data_users_valid = data_users_valid['is_ultra_corrected']

## 3. Исследование моделей машинного обучения

#### Модели случайного леса

Для неисправленных тарифов, то есть для ключевого признака **is_ultra**, рассмотрим модели случайного леса со следующим диапазоном гиперпараметров:  
* количество деревьев **n_estimators** от 1 до 50  
* максимальная глубина **max_depth** = 10  
  
и выберем лучшую модель.

In [14]:
table = np.zeros((50, 10, 1))
for i in range(50): #Комментарий наставника: чтобы не прибавлять единицу каждый раз, можно так: in range(1, 51)
    for j in range (10): #in range (1, 11)
        model_forest = RandomForestClassifier(n_estimators=i+1, max_depth=j+1, random_state=12345)
        model_forest.fit(features_data_users_trainig, target_data_users_trainig)
        predictions = model_forest.predict(features_data_users_valid)
        table[i][j][0] = accuracy_score(target_data_users_valid , predictions)
print('max accuracy =', np.amax(table))
print('положение max accuracy:', np.argmax(table))

max accuracy = 0.7916018662519441
положение max accuracy: 359


Лучшая точночть (79%) у модели с гиперпараметрами **n_estimators** = 36 и **max_depth** = 10

В данном случае гиперпараметры из **положение max accuracy** вычисляются следующим образом:  
* 359 делится с остатком на 10, получается 35 и остаток 9  
* **n_estimators** получается как 35+1, а **max_depth** - как 9+1

Для исправленных тарифов, то есть для ключевого признака **is_ultra_corrected**, рассмотрим модели случайного леса с таким же диапазоном гиперпараметров

In [16]:
table = np.zeros((50, 10, 1))
for i in range(50):
    for j in range (10):
        model_forest = RandomForestClassifier(n_estimators=i+1, max_depth=j+1, random_state=12345)
        model_forest.fit(features_data_users_trainig, c_target_data_users_trainig)
        predictions = model_forest.predict(features_data_users_valid)
        table[i][j][0] = accuracy_score(c_target_data_users_valid , predictions)
print('max accuracy =', np.amax(table))
print('положение max accuracy:', np.argmax(table))

max accuracy = 0.9035769828926905
положение max accuracy: 148


Лучшая точночть (90%) у модели с гиперпараметрами **n_estimators** = 15 и **max_depth** = 9

#### Модели логической регрессии

Рассмотрим модель логической регрессии для неисправленных тарифов, то есть для ключевого признака **is_ultra**.

In [17]:
model = LogisticRegression(random_state=12345)
model.fit(features_data_users_trainig, target_data_users_trainig)
predictions = model.predict(features_data_users_valid)
accuracy = accuracy_score(target_data_users_valid , predictions)
print('accuracy =', accuracy)

accuracy = 0.7247278382581649


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Точность у такой модели составила 72%

Рассмотрим модель логической регрессии для исправленных тарифов, то есть для ключевого признака **is_ultra_correctad**.

In [18]:
model = LogisticRegression(random_state=12345)
model.fit(features_data_users_trainig, c_target_data_users_trainig)
predictions = model.predict(features_data_users_valid)
accuracy = accuracy_score(c_target_data_users_valid , predictions)
print('accuracy =', accuracy)

accuracy = 0.8164852255054432


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Точность у модели составила 81.6%

### Вывод

Лучше всего себя показала модель случайного леса с исправленными тарифами, то есть с ключевым признаком **is_ultra_corrected**: точность составила 90%. Гиперпараметры: **n_estimators** = 15 и **max_depth** = 9.   
Если рассматривать неисправленные тарифы, то есть для ключевого признака **is_ultra_corrected**, то лучшей оказалась модель случайного леса с гиперпараметрами **n_estimators** = 36 и **max_depth** = 10. Точность соствила 79%, в то время как у модели логической регрессии точность составила лишь 72%.

## 4. Проверка моделей на тестовой выборке

Проверим модели случайного леса на тестовых выборках, с исправленными данными и без исправлений.

Модель случайного леса с ключевым признаком **is_ultra** и параметрами **n_estimators** = 36 и **max_depth** = 10

In [19]:
model_forest_36_10 = RandomForestClassifier(n_estimators=36, max_depth=10, random_state=12345)
model_forest_36_10.fit(features_data_users_tr_val, target_data_users_tr_val)
predictions = model_forest_36_10.predict(features_data_users_test)
accuracy = accuracy_score(target_data_users_test , predictions)
print('accuracy =', accuracy)

accuracy = 0.8009331259720062


Точность составила 80%

Модель случайного леса с ключевым признаком **is_ultra_corrected** и параметрами **n_estimators** = 15 и **max_depth** = 9

In [20]:
model_forest_15_9_c = RandomForestClassifier(n_estimators=15, max_depth=9, random_state=12345)
model_forest_15_9_c.fit(features_data_users_tr_val, c_target_data_users_tr_val)
predictions = model_forest_15_9_c.predict(features_data_users_test)
accuracy = accuracy_score(c_target_data_users_test , predictions)
print('accuracy =', accuracy)

accuracy = 0.9253499222395023


Точность составила 92.5%

### Вывод
Точность модели, использовавшей исправленные данные, составила 92.5%.  
Точность модели, использовавшей изначальные данные, составила 80%.

## 5. Проверка моделей на адекватность

Простая проверка на адекватность: предложить всем абонентом перейти на тариф smart

In [21]:
# сколько угадаем, если всем предложить тариф smart
print('точность:', data_users.is_ultra.value_counts()[0] / len(data_users))

точность: 0.693528313627878


В таком случае почти в 70% случаев это окажется верным предложением.  
А модель случайного леса будет давать результат на 10% лучше, что, конечно, лучше.

Если же выбор тарифов исправить, то и точность в случае предложения всем одного тарифа упадет до 59%, и точность модели увеличится до 92.5%

In [22]:
print('точность:', data_users.is_ultra_corrected.value_counts()[0] / len(data_users))

точность: 0.5933416303671437


## 6. Вывод

Лучше всего себя показала модель случайного леса с исправленными тарифами, то есть с ключевым признаком **is_ultra_corrected**: точность составила 92.5%. Гиперпараметры: **n_estimators** = 15 и **max_depth** = 9. Её и можно использовать определения, какой новый тариф предложить пользователям архивных тарифов. 